In [1]:
train_file = "./data/messages.json"

In [2]:
import pandas as pd

In [3]:
df = pd.read_json(train_file)

In [4]:
df.shape

(517311, 7)

In [5]:
# senders = set(df["sender"].value_counts()[:300])
senders = set(df["sender"].sample(n=10))
df1 = df[df.sender.isin(senders)]

In [6]:
len(senders)

10

In [7]:
df = df1

In [8]:
len(df["sender"].unique())

10

In [9]:
df.shape

(3318, 7)

In [10]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatiser = WordNetLemmatizer()

In [11]:
# Importing necessary libraries
from sklearn.preprocessing import LabelEncoder
y = df["sender"]
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [12]:
y

array([0, 0, 0, ..., 4, 0, 0])

In [13]:
X = df["message"]

In [14]:
import time
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=int(time.time()))

In [15]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2654,) (664,) (2654,) (664,)


In [16]:
# Importing necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer=CountVectorizer(stop_words="english", ngram_range=(1, 3)).fit(X_train)
text_bow_train=bow_transformer.transform(X_train)
text_bow_test=bow_transformer.transform(X_test)

In [17]:
names = bow_transformer.get_feature_names()
print(len(names), names[:5])

311473 ['00', '00 00', '00 00 00', '00 00 cancelled', '00 00 central']


In [18]:
text_bow_train

<2654x311473 sparse matrix of type '<class 'numpy.int64'>'
	with 1021222 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True, sublinear_tf=True)

In [20]:
wc_vector_train = text_bow_train

In [21]:
tfidf_fitted_transformer = tfidf_transformer.fit(wc_vector_train)

In [22]:
tfidf_train = tfidf_fitted_transformer.transform(wc_vector_train)

In [23]:
tfidf_train

<2654x311473 sparse matrix of type '<class 'numpy.float64'>'
	with 1021222 stored elements in Compressed Sparse Row format>

In [24]:
wc_vector_test = text_bow_test

In [25]:
tfidf_test = tfidf_fitted_transformer.transform(wc_vector_test)

In [26]:
# Importing necessary libraries
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=0.1, fit_prior=True).fit(text_bow_train, y_train)

In [27]:
model.score(text_bow_train, y_train)

0.9943481537302186

In [28]:
model.score(text_bow_test, y_test)

0.9653614457831325

In [29]:
# Importing necessary libraries
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB(alpha=0.1, fit_prior=True).fit(tfidf_train, y_train)

In [30]:
model1.score(tfidf_train, y_train)

0.9928409947249435

In [31]:
model1.score(tfidf_test, y_test)

0.9487951807228916